In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
import logistic_regression_util

import sklearn.linear_model
import sklearn.model_selection

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz


import graphviz
from graphviz import Graph

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

import acquire
import prepare


## Mini Exercise

1. Load the titanic dataset that you've put together from previous lessons.
2. Split your data into training and test.
3. Fit a logistic regression model on your training data using sklearn's
   linear_model.LogisticRegression class. Use fare and pclass as the
   predictors.
4. Use the model's `.predict` method. What is the output?
5. Use the model's `.predict_proba` method. What is the output? Why do you
   think it is shaped like this?
6. Evaluate your model's predictions on the test data set. How accurate
   is the mode? How does changing the threshold affect this?

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

titanic = sns.load_dataset('titanic')[['fare', 'pclass', 'survived']]
train, test = train_test_split(titanic, random_state=123, train_size=.8)

X = train[['fare', 'pclass']]
y = train.survived

model = LogisticRegression(random_state=123).fit(X, y)

In [ ]:
# unique values from the y variable
model.classes_

In [ ]:
pd.DataFrame(model.predict_proba(X), columns=model.classes_)

In [ ]:
train['yhat'] = model.predict(X)
train['p_survived'] = model.predict_proba(X)[:, 1]

In [ ]:
model.score(X, y)

In [ ]:
accuracy_score(train.survived, train.yhat)

In [ ]:
precision_score(train.survived, train.yhat, average=None)

In [ ]:
recall_score(train.survived, train.yhat)

In [ ]:
t = .25
train['yhat'] = train.p_survived > t

accuracy_score(train.survived, train.yhat), precision_score(train.survived, train.yhat), recall_score(train.survived, train.yhat)

## More Complicated Example

- **validate data split** lets us compare models, tweak hyperparams, experiment with thresholds without leaking information from the test split
- train: fit models -- majority of our data
- validate: compare models, choose hyperparams, thresholds -- ~ 20% of our data
- test: to get an idea of *out of sample error* -- ~20% of our data

In [ ]:
df = sns.load_dataset('titanic')[['fare', 'sex', 'pclass', 'survived']]
df.head()

In [ ]:
train, test = train_test_split(df, random_state=123, train_size=.86)
train, validate = train_test_split(train, random_state=123, train_size=.83)

print('    test: %d rows x %d columns' % test.shape)
print('   train: %d rows x %d columns' % train.shape)
print('validate: %d rows x %d columns' % validate.shape)

In [ ]:
model = smf.logit('survived ~ fare + sex + pclass', train).fit()
model.summary()

In [ ]:
probs = model.predict(train)
actual = train.survived

In [ ]:
validate.survived.mean()

In [ ]:
logistic_regression_util.plot_true_by_probs(actual, probs, subplots=True)

In [ ]:
logistic_regression_util.plot_metrics_by_thresholds(actual, probs)

In [ ]:
t = .55
probs = model.predict(test)
yhat = (probs > t).astype(int)
actual = test.survived

accuracy_score(actual, yhat)

# Logistic Regression Exercise

In this exercise, we'll continue working with the titanic dataset and building logistic regression models. Throughout this exercise, be sure you are training, evaluation, and comparing models on the train and validate datasets. The test dataset should only be used for your final model.

In [ ]:
titanic = acquire.get_titanic_data()

In [ ]:
def encode_sex(df):
    '''
    Returns a new dataframe with the ``sex`` column encoded.
    '''
    return df.assign(
        sex=(df.sex == 'female').astype(int)
    )

In [ ]:
def get_splits(titanic):
    '''
    Returns X and y for train, validate and test datasets
    '''
    # don't blow away our original data
    titanic = titanic.copy()
    
    # ignore warnings just for this block
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        scaler, encoder, train, test = prepare.prep_titanic(titanic)
    
    # Which features are we going to look at?
    cols = ['survived', 'pclass', 'sex', 'age', 'alone']
    train = train[cols]
    test = test[cols]

    # validate data split
    train, validate = sklearn.model_selection.train_test_split(
        train, train_size=.85, random_state=123
    )

    # split into X and y
    X_train, y_train = train.drop(columns='survived'), train.survived
    X_validate, y_validate = validate.drop(columns='survived'), validate.survived
    X_test, y_test = test.drop(columns='survived'), test.survived
    
    X_train = encode_sex(X_train)
    X_validate = encode_sex(X_validate)
    X_test = encode_sex(X_test)
    
    return X_train, y_train, X_validate, y_validate, X_test, y_test

In [ ]:
X_train, y_train, X_validate, y_validate, X_test, y_test = get_splits(titanic)

print('   train: %d rows' % X_train.shape[0])
print('validate: %d rows' % X_validate.shape[0])
print('    test: %d rows' % X_test.shape[0])

### Compare Models

In [ ]:
# a dataframe to hold our models' predictions for future comparison
evaluation = pd.DataFrame({
    'actual': y_validate
})

### Model Creation

Note that I've built my notebook such that one cell is one "unit" of model creation.
At the top of every cell I re-call `get_splits` so that I'm free to screw around with the data (e.g. dropping, renaming columns) within that cell, but it will be reset for the next one.
The last step of each cell is storing the model's predictions.

In [ ]:
# survived ~ pclass + age
X_train, y_train, X_validate, y_validate, X_test, y_test = get_splits(titanic)
X_train = X_train.drop(columns=['alone', 'sex'])
X_validate = X_validate.drop(columns=['alone', 'sex'])

model = sklearn.linear_model.LogisticRegression()
model.fit(X_train, y_train)
# [:, 1] -- numpy matrix -- all the rows, just the second column
evaluation['survived ~ pclass + age'] = model.predict_proba(X_validate)[:, 1]

In [ ]:
# survived ~ pclass + age + sex
X_train, y_train, X_validate, y_validate, X_test, y_test = get_splits(titanic)
X_train = X_train.drop(columns=['alone'])
X_validate = X_validate.drop(columns=['alone'])

model = sklearn.linear_model.LogisticRegression()
model.fit(X_train, y_train)
evaluation['survived ~ pclass + age + sex'] = model.predict_proba(X_validate)[:, 1]

In [ ]:
# survived ~ pclass + age + sex + alone
X_train, y_train, X_validate, y_validate, X_test, y_test = get_splits(titanic)

model = sklearn.linear_model.LogisticRegression()
model.fit(X_train, y_train)
evaluation['survived ~ pclass + age + sex + alone'] = model.predict_proba(X_validate)[:, 1]

In [ ]:
evaluation

- A threshold is a value we choose
- if the probability the model gives us is above the threshold, predict positive
- if the probability the model gives us is below the threshold, predict negative
- `.predict` -- makes predictions with a threshold of .5
- When the threshold is 0, predict everything as positive (i.e. 1)
- When the threshold is 1, predict everything as negative (i.e. 0)

### Evaluation

In [ ]:
logistic_regression_util.plot_metrics_by_thresholds(
    evaluation.actual, evaluation['survived ~ pclass + age']
)

In [ ]:
logistic_regression_util.plot_metrics_by_thresholds(
    evaluation.actual, evaluation['survived ~ pclass + age + sex']
)

In [ ]:
logistic_regression_util.plot_metrics_by_thresholds(
    evaluation.actual, evaluation['survived ~ pclass + age + sex + alone']
)

It looks like the model with more features is better here.

### Choosing a Threshold

Things to consider:

- where is an acceptable balance of the metrics?
- If I'm optimizing for precision/recall, where is a point where accuracy is still good, and the other metrics aren't terrible?
- What is the cost of a false positive vs false negative?

In [ ]:
thresholds = logistic_regression_util.evaluate_thresholds(evaluation.actual, evaluation['survived ~ pclass + age + sex + alone'])
thresholds.sort_values(by='accuracy')

In [ ]:
t = .66

predictions = (evaluation['survived ~ pclass + age + sex + alone'] > t).astype(int)
actual = evaluation.actual

# confusion matrix
pd.crosstab(predictions, actual, normalize=True)

### Exploring the `C` Hyperparameter

Two things:

1. The `C` hyperparameter: can constrain the size of the coefficients
2. The python code

Takeaways:

1. Try experimenting with differing values of `C` on the validate split
2. Choose a lower value for `C` to reduce overfitting (too low of a value might lead to underfitting)
3. If `C` is important, scale your data (regularization works best with scaled data)

In [ ]:
X_train, y_train, X_validate, y_validate, X_test, y_test = get_splits(titanic)

# TODO: allow for a threshold
# TODO: include precision and recall
def evaluate_model(c):
    model = sklearn.linear_model.LogisticRegression(C=c)
    model.fit(X_train, y_train)
    accuracy = model.score(X_validate, y_validate)
    coefs = dict(zip(X_train.columns, model.coef_[0]))
    return {'C': c, 'accuracy': accuracy, **coefs}

models = [evaluate_model(c) for c in [.001, .01, .1, 1, 10, 100, 1000]]
(pd.DataFrame(models).round(3)
 .set_index(['C', 'accuracy'])
 .style
 .set_caption('Effect of differnt C values on accuracy (t=.5) and the resulting coefficients.')
 .set_precision(3)
#  .background_gradient('Blues')
#  .highlight_max() # for columns
#  .highlight_max(axis=1) # for rows
)

In [ ]:
dict(zip(X_train.columns, model.coef_[0]))

5. **Bonus** How do different strategies for handling the missing values in the age column affect model performance?

6. **Bonus**: How do different strategies for encoding sex affect model performance?

7. **Bonus**: scikit-learn's LogisticRegression classifier is actually applying a regularization penalty to the coefficients by default. This penalty causes the magnitude of the coefficients in the resulting model to be smaller than they otherwise would be. This value can be modified with the C hyper parameter. Small values of C correspond to a larger penalty, and large values of C correspond to a smaller penalty.

    Try out the following values for C and note how the coefficients and the model's performance on both the dataset it was trained on and on the validate split are affected.
                                C=.01,.1,1,10,100,1000

**Bonus Bonus**: how does scaling the data interact with your choice of C?

# Decision Tree Exercise

In [51]:
df = sns.load_dataset("titanic")
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [52]:
# drop duplicate columns
# Drop columns that we discovered from Explore stage didn't really have a lot of bearing
df = df[["survived", "pclass", "sex", "age", "fare"]]
df.head()

,survived,pclass,sex,age,fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500


In [53]:
def encode_gender(gender):
    if gender == "male":
        return 0
    else:
        return 1

In [54]:
df.sex = df.sex.apply(encode_gender)
df.head()

,survived,pclass,sex,age,fare
0,0,3,0,22.0,7.2500
1,1,1,1,38.0,71.2833
2,1,3,1,26.0,7.9250
3,1,1,1,35.0,53.1000
4,0,3,0,35.0,8.0500


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   survived  891 non-null    int64  
 1   pclass    891 non-null    int64  
 2   sex       891 non-null    int64  
 3   age       714 non-null    float64
 4   fare      891 non-null    float64
dtypes: float64(2), int64(3)
memory usage: 34.9 KB


In [56]:
print(f"Survived nulls: {df.survived.isna().sum()}")
print(f"Class nulls:  {df.pclass.isna().sum()}")
print(f"Gender nulls: {df.sex.isna().sum()}")
print(f"Age nulls: {df.age.isna().sum()}")
print(f"Fare nulls: {df.fare.isna().sum()}")

Survived nulls: 0
Class nulls:  0
Gender nulls: 0
Age nulls: 177
Fare nulls: 0


In [57]:
# nice and clean
df.isna().sum()

survived      0
pclass        0
sex           0
age         177
fare          0
dtype: int64

In [58]:
# get the median age
median_age = df[df.age.notnull()].age.median()
median_age

28.0

In [59]:
# the pandas .median method ignores nulls
df.age.median()

28.0

In [60]:
# fill the nulls w/ the median
df.age = df.age.fillna(median_age)
print(f"Age nulls: {df.age.isna().sum()}")

Age nulls: 0


In [61]:
# Setup the X and y variables
X = df.drop("survived", axis=1)
y = df[["survived"]]

In [62]:
# Setup the train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state = 123)

In [63]:
# for classification you can change the algorithm to gini or entropy (information gain).  Default is gini.
clf = DecisionTreeClassifier(criterion='entropy', max_depth=8, random_state=123)

1. Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

In [64]:
clf.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=8, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=123, splitter='best')

In [65]:
y_pred = clf.predict(X_train)
y_pred

array([1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0,

### Model's Predicted Performance

In [66]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))

Accuracy of Decision Tree classifier on training set: 0.90


In [67]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.95      0.92       379
           1       0.91      0.83      0.87       244

    accuracy                           0.90       623
   macro avg       0.90      0.89      0.90       623
weighted avg       0.90      0.90      0.90       623



### Model Performance on Test Data

In [68]:
# Get the predicted y values from the X_test
y_pred = clf.predict(X_test)

In [69]:
print(f"Accuracy of Decision Tree on Test data is: {clf.score(X_test, y_test)}")

Accuracy of Decision Tree on Test data is: 0.8283582089552238


In [70]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.87      0.87       170
           1       0.77      0.76      0.76        98

    accuracy                           0.83       268
   macro avg       0.82      0.81      0.81       268
weighted avg       0.83      0.83      0.83       268



In [71]:
dot_data = export_graphviz(clf, out_file=None) 
graph = graphviz.Source(dot_data) 

graph.render('iris_decision_tree', view=True)

'iris_decision_tree.pdf'